In [1]:
import os
import numpy as np
from pyLAMMPS import LAMMPS_input, get_molecule_coordinates
from tools.utils import work_json

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.


In [10]:
from typing import List, Tuple
from feos.pcsaft import PcSaftParameters
from feos.eos import EquationOfState, PhaseDiagram
from feos.si import KELVIN, BAR, PASCAL, KILOGRAM, METER
from feos.gc_pcsaft import IdentifierOption

def get_initial_conditions(SAFT_parameter_file: str, molecule_smiles: List[str], compositions: List[float], temperature: float=None, pressure: float=None, 
                           SAFT_binary_file: str="", n_eval_saft: int=51, verbose: bool=False) -> Tuple[List, List, List, List]:
    """
    Function that uses the feos implementation of the PC-SAFT equation of state to determine suitable starting temperatures, pressures and densities for any mixture.

    Args:
        SAFT_parameter_file (str): File containing the PC-SAFT parameters
        molecule_smiles (List[str]): SMILES of each molecule in the mixture
        compositions (List[float]): Compositions that should be extracted of this mixture
        temperature (float, optional): Constant temperature of the mixture. If constant pressure should be used, temperature should equal None. Defaults to None.
        pressure (float, optional): Constant pressure of the mixture. If constant temperature should be used, pressure should equal None. Defaults to None.
        SAFT_binary_file (str, optional): File containing binary PC-SAFT parameters. Defaults to "".
        n_eval_saft (int, optional): Evaluation steps of the PC-SAFT VLE object. Defaults to 51.
        verbose (bool, optional): If the binary parameter should be printed out. Defaults to False.


    Returns:
        temperatures (List): Temperatures of the mixture at the specified compositions.
        pressures (List): Pressures of the mixture at the specified compositions.
        densities (List): Densities of the mixture at the specified compositions.
        activity_coeff (List): Symmetric activity coefficients at the specified compositions. (For each composition it contains gamma_i and gamma_j)
    """
    
    # Get SAFT paramters
    if SAFT_binary_file:
        parameters = PcSaftParameters.from_json( molecule_smiles, SAFT_parameter_file, SAFT_binary_file, search_option = IdentifierOption.Smiles )
    else:
        parameters = PcSaftParameters.from_json( molecule_smiles, SAFT_parameter_file, search_option = IdentifierOption.Smiles )

    if verbose:
        print("Binary parameter: k_ij = %f\n"%parameters.k_ij[0][1])

    # Call the PC-SAFT equation of state class
    pc_saft = EquationOfState.pcsaft( parameters )

    # Get a VLE of either constant temperature or pressure
    key     = temperature * KELVIN if temperature else pressure * BAR
    vle     = PhaseDiagram.binary_vle( eos = pc_saft, temperature_or_pressure = key, npoints = n_eval_saft)

    # Extract temperatures, pressures, and densities at the specified compositions
    saft_compositions = [ np.round( state.liquid.molefracs[0], 3).tolist() for state in vle.states if np.isin(round(state.liquid.molefracs[0],3),compositions)] 
    temperatures      = [ np.round( state.liquid.temperature / KELVIN, 3).tolist() for state in vle.states if np.isin(round(state.liquid.molefracs[0],3),compositions) ]
    pressures         = [ np.round( state.liquid.pressure() / PASCAL * 1e-5, 3).tolist() for state in vle.states if np.isin(round(state.liquid.molefracs[0],3),compositions) ]
    densities         = [ np.round( state.liquid.mass_density() / ( KILOGRAM / METER**3 ), 2).tolist() for state in vle.states if np.isin(round(state.liquid.molefracs[0],3),compositions) ]
    activity_coeff    = [ np.round( np.exp(state.liquid.ln_symmetric_activity_coefficient()), 3).tolist() for state in vle.states if np.isin(round(state.liquid.molefracs[0],3),compositions) ]

    if not saft_compositions == compositions:
        raise KeyError("Not all specified compositions are found in the VLE object. Increase the number of VLE evaluations!\n")
    
    return temperatures, pressures, densities, activity_coeff


def create_composition_matrix(no_components: int, composition: List) -> List[ List[ List ] ]:
    """
    Function that makes a compositon matrix for an arbitrary mixture.

    Args:
        no_components (int): Number of componets in the mixture.
        composition (List): Composition that should be simulated for each pure component.

    Returns:
        composition_matrix List[ List[ List ] ]: Composition matrix. [ [x1=[0.0, ... 1.0], x2, x3, ...], [x1, x2=[0.0, ..., 1.0], x3, ... ], ... ]
    """
    # Create empty composition matrix
    composition_matrix = [ [] for _ in range(no_components) ]

    # Fill the diagonal with the original composition and off-diagonal with complementary composition
    for i in range(no_components):
        for j in range(no_components):
            composition_matrix[i].append( composition if i == j else (1 - np.array(composition)).tolist() )

    return composition_matrix


# System setup

This notebook sets up the systems needed to compute the vapor liquid equilibrium for a specified mixture using the solvation free energy. The solvation free energy is computed using the decoupling approach and LAMMPS. For every component, the self-solvation free energy, as well as the solvation free energy in several mixture compositions is needed.

Essentially, the workflow can be summarized as follows:
1. Get intial molecule coordinates using SMILES and PubChem and provide a graph representation.
2. Use PLAYMOL to construct a system of any mixture.
3. Use the moleculegraph software and pyLAMMPS tools to generate LAMMPS data and input files via jinja2 templates.

## 1. Define general settings ##

1. Define paths to force field toml (in a moleculegraph understandable format), as well as to all jinja2 templates.
2. Define name, SMILES, and graph strings of the molecules under investigation. (further examples on constructing molecule graphs available at https://github.com/maxfleck/moleculegraph)
3. Define free energy settings (coupling lambdas for each species for each interaction type (van der Waals or Coulomb))

In [3]:
# 1: Define path to toml force field, SAFT files and jinja2 templates. Also define a databank file, containing all important information of the system
# Path to force field toml
force_field_path     = "input_files/force-fields/forcefield_TAMie_UA.toml"

# Path to SAFT related files
SAFT_parameter_file  = "input_files/SAFT/SI_pcp-saft_parameters.json"
SAFT_binary_file     = "input_files/SAFT/binary_records_kij.json"

# Path to xyz template
template_xyz         = "input_files/templates/template_write_xyz.xyz"

# Path to playmol templates
playmol_force_field_template = "input_files/templates/template_playmol_forcefield.playmol"
playmol_input_template = "input_files/templates/template_playmol_input.mol"

# Path to LAMMPS templates
LAMMPS_data_template  = "input_files/templates/template_lammps_data.data"
LAMMPS_input_template = "input_files/templates/template_lammps_free_energy.in"

# Path to mixture databank and definition of dataset for this mixture
databank_json          = "input_files/mixture_dataset.json"
mixture_dataset       = {}

# 2: Names, SMILES, and graphs of molecules (further examples on constructing molecule graphs available at https://github.com/maxfleck/moleculegraph)
# Define name of the mixture
system_name           = "mixture_hexane_butylamine"

# Define unique condition of the mixture (e.g: a constant temperature or a constant pressure, as well as the compositions of the N-1 components at each statepoint. )
# Also specify the number of molecules that should be simulated in total and if the molecules posses partial charges.
system_key            = "373"
temperature           = 373.15
pressure              = None
compositions          = [ 1.0 ]
total_no_molecules    = 900

# Define every component in the mixture
molecule_name1        = "hexane"
molecule_graph1       = "[CH3_alkane][CH2_alkane][CH2_alkane][CH2_alkane][CH2_alkane][CH3_alkane]"
molecule_smiles1      = "CCCCCC"
molecule_charge1      = False

molecule_name2        = "butylamine"
molecule_graph2       = "[CH3_alkane][CH2_alkane][CH2_alkane][CH2_amine][NH2_amine][b1][cH_amine][cH_amine]"
molecule_smiles2      = "CCCCN"
molecule_charge2      = True

# Define the whole mixture list
molecule_names_list   = [ molecule_name1, molecule_name2 ]
molecule_graphs_list  = [ molecule_graph1, molecule_graph2 ]
molecule_smiles_list  = [ molecule_smiles1, molecule_smiles2 ]
molecule_charges_list = [ molecule_charge1, molecule_charge2 ]

# ********************* To do: fix that composition matrix is not only for 2 components *********************
# Define composition matrix [ [x1=[0.0, ... 1.0], x2, x3, ...], [x1, x2=[0.0, ..., 1.0], x3, ... ], ... ]
# This matrix contain per element a sublist with the concentration entries of all components. On the diagonal are always the concentrations of the component starting from 0.0 ranging to 1.0
compositions          = create_composition_matrix( len(molecule_names_list), compositions )

# 3: Define free energy settings (coupling lambdas for each species for each interaction type (van der Waals or Coulomb))
# Define coupling pair style for van der Waals and for Coulomb interactions
rcut                  = 14
pair_style_coupling_vdw     = f"hybrid/overlay mie/cut {rcut} mie/cut/soft 1.0 0.5 {rcut}"
pair_style_coupling_coulomb = f"coul/long {rcut} coul/cut/soft 1.0 0.0 {rcut}"

# Define special bonds for 1-2, 1-3, and 1-4 interactions
sb_dict               = {"vdw":[0,0,0],"coulomb":[0,0,0]}

# Define which force field types should be constrained using the SHAKE algorithm
shake_dict            = {"atoms":[],"bonds":[],"angles":[]}

# Define the free energy method that should be utilized
free_energy_method    = "TI"

# Define the coupling lambdas that should be utilized for each component (for vdW and Coulomb. If a component is uncharged, leave the Coulomb list empty)
l_test = [ 0.0   , 0.0125, 0.025 , 0.0375, 0.05  , 0.0625, 0.075 , 0.0875,
           0.1   , 0.1125, 0.125 , 0.1375, 0.15  , 0.1625, 0.175 , 0.1875,
           0.2   , 0.2125, 0.225 , 0.2375, 0.25  , 0.2625, 0.275 , 0.2875,
           0.3   , 0.3125, 0.325 , 0.3375, 0.35  , 0.3625, 0.375 , 0.3875,
           0.4   , 0.4125, 0.425 , 0.4375, 0.45  , 0.4625, 0.475 , 0.4875,
           0.5   , 0.5125, 0.525 , 0.5375, 0.55  , 0.5625, 0.575 , 0.5875,
           0.6   , 0.6125, 0.625 , 0.6375, 0.65  , 0.6625, 0.675 , 0.6875,
           0.7   , 0.7125, 0.725 , 0.7375, 0.75  , 0.7625, 0.775 , 0.7875,
           0.8   , 0.8125, 0.825 , 0.8375, 0.85  , 0.8625, 0.875 , 0.8875,
           0.9   , 0.9125, 0.925 , 0.9375, 0.95  , 0.9625, 0.975 , 0.9875,
           1.0   ]

lambdas_coupling     = [ [ [ *l_test ], [] ], 
                         [ [ *l_test ], [] ] ]


# Write general information in the mixture databank
mixture_dataset["general_information"] = { "system_name": system_name, "molecule_names_list": molecule_names_list, "molecule_graphs_list": molecule_graphs_list,
                                           "molecule_smiles_list": molecule_smiles_list, "molecule_charges_list": molecule_charges_list, "compositions": compositions,
                                           "pair_style_coupling_vdw": pair_style_coupling_vdw, "pair_style_coupling_vdw": pair_style_coupling_vdw,
                                           "pair_style_coupling_coulomb":pair_style_coupling_coulomb, "sb_dict": sb_dict, "shake_dict": shake_dict }

# Write free energy settings
mixture_dataset["free_energy"]         = { "free_energy_method": free_energy_method, "lambdas_coupling": lambdas_coupling }

## 2. Build each needed mixture system 

Build the systems to simulate the self-solvation energy of each molecule, as well as the mixture solvation free energies.

In [15]:
# Add thermodynamic properties to mixture dataset
mixture_dataset["thermodynamic_settings"] = {}

# Decide if PLAYMOL should build system 
build_playmol = False

# Loop through the components and build each system and write LAMMPS data and input files
for i, molecule_name in enumerate(molecule_names_list):

    print(f"\nInsertion of {molecule_name}\n")

    # Get the mixture conditions with PC-SAFT based on the molefractions of the current molecule in the iteration. 
    # (Hence, pass always the smiles of the current component as first entry)
    remaining_idx = [index for index, _ in enumerate(molecule_smiles_list) if index != i]
    
    remaining_names   = np.array(molecule_names_list)[remaining_idx].tolist()
    remaining_graphs  = np.array(molecule_graphs_list)[remaining_idx].tolist()
    remaining_smiles  = np.array(molecule_smiles_list)[remaining_idx].tolist()
    remaining_charges = np.array(molecule_charges_list)[remaining_idx].tolist()
    
    temperatures, pressures, densities, activity_coeff = get_initial_conditions( SAFT_parameter_file = SAFT_parameter_file, molecule_smiles = [molecule_smiles_list[i], *remaining_smiles ], 
                                                                                 compositions = compositions[i][i], temperature = temperature, pressure = pressure )
    
    # Save estimated starting conditions in dataset
    mixture_dataset["thermodynamic_settings"][molecule_name] =  { "temperatures": temperatures, "pressures": pressures, "densities": densities, "activity_coeff":activity_coeff }

    for j, (xi, temp, press, dens) in enumerate( zip( compositions[i][i], temperatures, pressures, densities ) ):

        print(f"\nState point: xi = {xi}, T = {temp}, p = {press}, rho_mass = {dens}\n")

        # Differentiate between self-solvation free energy (at xi = 1.0) and solvation free energy in mixture
        mol_name_list    = [ molecule_name + "_coupled", molecule_name, *(remaining_names if xi < 1.0 else []) ]
        mol_graph_list   = [ molecule_graphs_list[i], molecule_graphs_list[i], *(remaining_graphs if xi < 1.0 else []) ]
        mol_smiles_list  = [ molecule_smiles_list[i], molecule_smiles_list[i], *(remaining_smiles if xi < 1.0 else []) ]
        mol_charges_list = [ molecule_charges_list[i], molecule_charges_list[i], *(remaining_charges if xi < 1.0 else []) ]

        # Define pair style for decoupling approach, if any component in the mixture is charged use Coulomb potential in LAMMPS.
        charged             = any( mol_charges_list)
        pair_style_coupling = f"{pair_style_coupling_vdw} {pair_style_coupling_coulomb}" if charged else f"{pair_style_coupling_vdw}"

        # Define system name
        sub_system       = "mix_" + "_".join(mol_name_list)

        # Path to working folder
        working_folder   = f"simulation_systems/{system_name}/{system_key}/{sub_system}"

        # Define output path for final xyz files
        xyz_destinations = [ f"{working_folder}/%s.xyz"%name for name in mol_name_list ]

        # Get the single molecule coordinates for each component (these are mixture dependent, as running atom numbers are introduced in the xyz files)
        get_molecule_coordinates( molecule_name_list = mol_name_list, molecule_graph_list = mol_graph_list, molecule_smiles_list = mol_smiles_list,
                                  xyz_destinations = xyz_destinations, template_xyz = template_xyz, verbose = False )
    
        # Call the LAMMPS input class
        LAMMPS_class        = LAMMPS_input( mol_str = mol_graph_list, ff_path = force_field_path, decoupling = True )

        # Write PLAYMOL force field file using the LAMMPS_input class
        playmol_force_field_destination = f"{working_folder}/playmol_ff.playmol"

        LAMMPS_class.prepare_playmol_input( playmol_template = playmol_force_field_template, playmol_ff_path = playmol_force_field_destination )

        # Create per mixture the composition folder (if not already done)
        composition_path = f"{working_folder}/x{xi}"
        os.makedirs( composition_path, exist_ok = True )

        # Get the molecule numbers according to the mixture composition (utilize closing condition for current component) 
        # Also split the number current molecules into 1 couple molecule and N1 - 1 molecules
        remaining_numbers = [ int( comp[j] * total_no_molecules ) for comp in np.array(compositions[i])[remaining_idx] ]
        molecule_numbers  = [ 1, total_no_molecules - sum(remaining_numbers)-1, *(remaining_numbers if sum(remaining_numbers) > 0 else []) ]
        
        # Prepare LAMMPS with molecules numbers and density of the system
        LAMMPS_class.prepare_lammps_data (nmol_list = molecule_numbers, densitiy = dens )

        # Write PLAYMOL input and execute using the LAMMPS_input class (if wanted.) --> the xyz will be in the same folder as the .mol
        # and will have the same name, just .xyz instead of .mol
        if build_playmol:
            playmol_input_destination = f"{composition_path}/{sub_system}.mol"
            playmol_relative_ff_path  = os.path.relpath(playmol_force_field_destination, os.path.dirname(playmol_input_destination))
            playmol_relative_xyz_path = [ os.path.relpath(xyz, os.path.dirname(playmol_input_destination)) for xyz in xyz_destinations ]

            LAMMPS_class.write_playmol_input( playmol_template = playmol_input_template, playmol_path = playmol_input_destination, 
                                              playmol_ff_path = playmol_relative_ff_path, xyz_paths = playmol_relative_xyz_path )

        # Write LAMMPS data file using the generated xyz file
        system_xyz              = f"{composition_path}/{sub_system}.xyz"
        LAMMPS_data_destination = f"{composition_path}/lammps.data"

        LAMMPS_class.write_lammps_data( xyz_path = system_xyz, data_template = LAMMPS_data_template, data_path = LAMMPS_data_destination )

        print("\nPrepare LAMMPS input files\n")
        
        # Write LAMMPS input file using the decoupling approach (perform an individual simulation for every lambda)
        for k,lambda_coupling in enumerate( lambdas_coupling[i] ):
            lambda_key = "vdw" if k == 0 else "coulomb"

            for l,lamdas in enumerate(lambda_coupling):
                
                # Define lambda for van der Waals / Coulomb interaction coupling
                lambda_vdw     = lamdas if k == 0 else 1.0
                lambda_coulomb = lamdas if k == 1 else 1e-9

                # Define the sampling pertubation depending on the free energy method
                dlambda        = [-0.001, 0.001] if free_energy_method == "TI" else \
                                 [ 0.0 if l == 0 else lambda_coupling[l-1] - lamdas, 0.0 if l == len(lambda_coupling)-1 else lambda_coupling[l+1] - lamdas ]

                # Define sampling output files 
                free_energy_output_files  = [ f"fep{l}{l-1}.sampling", f"fep{l}{l+1}.sampling" ]

                # Define LAMMPS input destination
                LAMMPS_input_destination  = f"{composition_path}/{lambda_key}/{free_energy_method}/sim_{l}/lammps.in"
                relative_LAMMPS_data_path = os.path.relpath(LAMMPS_data_destination, os.path.dirname(LAMMPS_input_destination))
    
                LAMMPS_class.prepare_lammps_input( pair_style = pair_style_coupling, 
                                                   sb_dict    = sb_dict,
                                                   shake_dict = shake_dict )

                LAMMPS_class.write_lammps_input( input_path = LAMMPS_input_destination, template_path = LAMMPS_input_template, data_file = relative_LAMMPS_data_path,
                                                 temperature = temp, pressure = press, equilibration_time = 2e6, production_time = 1e6,
                                                 free_energy_method = free_energy_method, lambda_vdw = lambda_vdw, lambda_coulomb = lambda_coulomb, dlambda = dlambda,
                                                 free_energy_output_files = free_energy_output_files )



# Save the dataset for this mixture in databank
work_json( file_path = databank_json, data = mixture_dataset, to_do = "append" )


Insertion of hexane


State point: xi = 1.0, T = 373.15, p = 2.46, rho_mass = 580.01


Prepare LAMMPS input files


Insertion of butylamine


State point: xi = 1.0, T = 373.15, p = 2.007, rho_mass = 676.19


Prepare LAMMPS input files

